<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 26 : Le Web, côté serveur</h1>

Nous avons vu qu'un navigateur Web peut envoyer à un serveur des requêtes **paramétrées**. Il existe deus modes de passage de paramètres, la méthode `GET` et la méthode `POST`.

## Concepts fondamentaux du Web côté serveur
### Etude de cas et principe de fonctionnement
Considérons un exemple concret.  
L'utilisatrice Alice navigue vers son un Web (un forum de discussion). Arrivée sur la page d'accueil, elle remplit un formulaire avec son identifiant et son mot de passe. Une fois authentifiée, elle peut poster des messages et accéder librement aux parties du site à accès restreint.  
Quand elle a terminé, Alice clique sur un bouton de déconnexion du site, elle se retrouve donc sur la page d'accueil. Les pages à accès restreint ne sont plus accessibles.  
Dans le même temps, sur une autre machine cliente, Bob se connecte lui aussi au site et s'identifie. Il utilise le site comme Alice et **en même temps qu'elle**. Leur vue du site n'est pas mélangée pour autant, le serveur répond à chacune des requêtes en envoyant le contenu attendu par chacun des clients.

Côté serveur, la plupart des langages de programmation Web fonctionne de la manière suivante :
1. Le serveur Web reçoit une requête HTTP demandant une certaine ressource (par exemple, `index.php?x=1&y=2`).
2. Le serveur détermine que la ressource ne doit pas être renvoyée telle quelle au client mais *évaluée*. Une manière de déterminer cela peut être l'extension du fichier (ici `.php`), mais ce n'est pas la seule manière de faire.
3. Le serveur appelle un programme externe (par exemple, l'interprète du langage PHP) pour évaluer le fichier. Il met à disposition de ce programme les paramètres de la requête ainsi que toutes les informations qui sont liées à cette dernière.
4. Le programme effectue un traitement en utilisant ces paramètres.
5. Le programme écrit (dans un fichier de sortie ou sur sa sortie standard) le contenu de la réponse de la requête. En général, c'est un document HTML, mais on peut aussi générer du CSV, du PDF, ...
6. Le serveur Web prend le fichier généré et le renvoie alors comme réponse au navigateur qui a fait la requête.

Dans ce modèle d'exécution, un programme est exécuté côté serveur pour chaque requête HTTP. Il produit une sortie (le résultat de la requête) puis **se termine**. 

Cette description ne permet pas de rendre compte de deux phénomènes :
1. Le site peut faire la différence entre Alice et Bob. Plus que ça, il peut personnaliser ses réponses en fonction de l'identité du client.
2. Plus important, entre deux requêtes distinctes, le serveur se souvient de l'état dans lequel était un client particulier lors de la dernière requête. Par exemple, tant qu'Alice ou Bob ne sont pas déconnectés, le serveur se souvient qu'ils se sont authentifiés un peu plus tôt et donc qu'ils peuvent accéder aux pages restreintes.

### Les cookies
Il est important pour un site Web de pouvoir **identifier** des différents clients (et les différencier). Cependant, il n'existe rien pour identifier un client de manière unique au niveau du réseau. Par exemple, l'adresse IP d'un client, bien qu'unique à un moment donné, n'est pas un identifiant valide :
* Plusieurs personnes, au sein d'un réseau local privé, peuvent partager la même IP publique (celle que le serveur voit). 
* Une adresse IP peut aussi changer. Par exemple, si un client est sur un réseau mobile, il peut être coupé du réseau momentanément à cause d'un obstacle, puis reconnecté. Il est alors possible que son adresse IP ait changé.

Une manière pur les sites Web de différencier leurs clients est l'utilisation de [**cookies**](https://www.cnil.fr/fr/cnil-direct/question/un-cookie-quest-ce-que-cest). Un cookie est une petite quantité de données. Il est composé d'un **nom**, d'une **valeur** et optionnellement d'une **date d'expiration**. Le nom, la valeur et la durée de vie sont choisis par le serveur. Lorsqu'un serveur veut déposer un cookie particulier chez un client, il l'envoie comme un entête de réponse HTTP.  
Par exemple, si un client demande une page `index.php`, le serveur peut répondre :

```
HTTP/1.1 200 OK
Date: Tue, 05 Jan 2021 12:48:46 GMT
Server: Apache
...
Set-Cookie: test=42; Expires=Mon, 30 May 2019 18:10:12 GMT;
...
Content-Length: 17843
Connection: close
Content-Type: text/html; charset=UTF-8
```
En recevant cette réponse du serveur, le navigateur Web voit que le serveur lui envoie un cookie, dont le nom est `test`, la valeur est `42` et la date d'expiration est le 30 mai 2019 à 18h10min12s.  
Le navigateur stocke le cookie, en l'associant au **nom de domaine** visité. Lorsque le navigateur effectue une nouvelle connexion, possiblement longtemps après, il renvoie le cookie au serveur dans sa requête, si la date d'expiration n'est pas dépassée.

```
GET /index.php HTTP/1.1
Host: ...
...
Cookie: test=42;
```

Ainsi, le serveur peut **enregistrer des données différentes** sur chaque client. Ces données sont ré-envoyées au serveur qui les a déposées. Cela permet au serveur de savoir qu'il a déjà vu ce client, et que la dernière fois que le client s'est connecté, le serveur lui a envoyé la valeur `42` pour le cookie `test`. Si la date d'expiration d'un cookie est passée, le navigateur Web le supprime. Si un cookie n'a pas de date, alors il est automatiquement supprimé lorsque le navigateur est fermé.

Ce mécanisme simple permet de stocker des informations de personnalisation. Par exemple, lorsque l'on visite un site traduit en plusieurs langues, le cookie peut stocker, dans un cookie, la dernière langue utilisée par un client, et afficher automatiquement le site dans cette langue lors d'une visite suivante.

Quelques exemples de cookies pour certains sites : 
* [www.cnil.fr](Fichiers/Cookies/cookies_cnil.json)
* [interstices.info](Fichiers/Cookies/cookies_interstices.json)
* [pixees.fr](Fichiers/Cookies/cookies_pixees.json)
* [www.lemonde.fr](Fichiers/Cookies/cookies_lemonde.json)
* [eduscol.education.fr](Fichiers/Cookies/cookies_eduscol.json)

### Session HTTP
Dans notre exemple initial, Alice a navigué sur la page d'accueil (une requête HTTP suivie d'une réponse), puis a navigué sur les pages restreintes (au moins une paire requête-réponse par page) puis a cliqué sur un bouton de déconnexion et s'est retrouvée sur la page d'accueil (nouvelle requête et réponse).  
Un tel ensemble de requêtes et réponse HTTP forment ce que l'on appelle une **session HTTP** .  
Une session commence par une phase de création de session, puis un ensemble de requêtes et réponses, puis se termine par une phase de destruction de session.  
L'intérêt est qu'une fois la session créée, le serveur se souvient des requêtes de la session, et ce, pour chaque client. Dans notre exemple cela signifie que le serveur se souvient à chaque requête HTTP qu'Alice et Bob sont authentifiés, sans qu'ils aient à rentrer leur mot de passe à chaque fois (ce qui rendrait la navigation pénible).

D'un point de vue technique, les sessions sont implémentées grâce aux cookies. Lorsqu'un client se connecte à un serveur, ce dernier regarde si le client lui fournit un cookie particulier (appelons le `ID`). Si ce n'est pas le cas, il crée une nouvelle valeur unique appelée identifiant de session (par exemple `12345`). Le serveur fait alors deux choses :
* il garde en mémoire dans un **dictionnaire global** une entrée associant l'identifiant de session (ici `12345`) à un dictionnaire de valeurs.
* le serveur renvoie aussi le code `ID=12345` au client. 

Lorsque le client se reconnecte, il renvoie son identifiant de session. Le serveur peut alors récupérer le dictionnaire de valeurs associé à l'identifiant et le passer au programme qui va générer la page Web. C'est de cette manière qu'on peut enregistrer **pour chaque client** des données qui restent côté serveur. Lorsqu'un client se déconnecte, le serveur retire juste l'entrée associée à l'identifiant et efface les valeurs conservées.

Cela explique pourquoi les sites contenant un formulaire d'authentification ne fonctionne pas lorsque les cookies sont désactivés dans les préférences du navigateur Web. Si le navigateur refuse de stocker les cookies, alors le serveur n'a plus le moyen d'obtenir un identifiant unique pour ce navigateur de façon fiable. 

Attention, le stockage des cookies est effectué par un navigateur pour s'authentifier sur un site et que l'on exécute en parallèle un autre navigateur pour naviguer vers le même site, on ne sera pas authentifié dans le second navigateur.

Lorsqu'on utilise le mode de navigation privée du navigateur, ce dernier supprime, lors de la fermeture de la fenêtre, tous les cookies qui ont été créés depuis le démarrage de la navigation privée, même s'ils n'ont pas expiré.

#### Cookies et vie privée
Les cookies ont été proposés comme un moyen technique au dessus duquel implémenter d'autres mécanismes tels que les sessions. L'une des règles de sécurité initiale, toujours en vigueur, est que les navigateurs peuvent envoyer un cookie **uniquement** au domaine qui l'a déposé initialement. Cette règle visait à interdire à des sites tiers *d'espionner* les utilisateurs en sachant quels sites sont visités et quelles valeurs sont dans les cookies (les dates, en particulier, peuvent donner une idée sur la fréquentation de tel ou tel site).  
Cependant cette règle peut facilement être contournée. Un site d'information, par exemple, `www.news.com` diffuse des articles variés. En dessous de chaque article, il propose des boutons **partager cet article sur `reseau-social.com` ou `micro-blogging.com`** (ou tout autre site). Les boutons contiennent des images, du code HTML et globalement des ressources accessibles à l'URL fournies par le site comme `https://www.reseau-social.com/bouton?origin=123`. Lorsqu'un utilisateur navigue sur `www.news.com` et clique sur l'un de ses boutons, son navigateur effectue une requête HTTP vers `https://www.reseau-social.com/bouton?origin=123`. Ce site est donc dans la situation suivante : 
* il peut légitimement déposer un cookie le concernant, car l'utilisateur fait une requête vers son domaine.
* il peut savoir quelle URL a été utilisée pour la requête, car c'est lui qui a fourni le code HTML à `www.news.com`. Il peut aisément connaître le site que l'utilisateur visitait, voire le contenu du site, au moment où l'utilisateur a partagé l'information. Il peut stocker toutes ces informations dans un cookie

Lorsque l'utilisateur visite `https://www.reseau-social.com`, ce dernier peut légitimement relire son cookie et afficher, par exemple, de la publicité en rapport avec l'article lu. Les navigateurs permettent de limiter cette pratique via l'option **interdire les cookies de tierce partie** : le navigateur refusera, alors, les cookies venant d'un domaine qui n'est pas celui affiché dans la barre d'adresse de l'URL.

Depuis mai 2018, le RGPD (Règlement Général sur la Protection des Données) encadre, entre autres, l'utilisation des cookies. Un utilisateur doit donner son consentement explicite et positif sur l'utilisation qu'un site fait des cookies, par catégories. Ainsi un utilisateur peut décider de n'accepter que les cookies *techniques* (ceux nécessaires à la mise en place d'une session) et refuser les autres cookies (publicité ciblée, télémétrie,...).

## Langage PHP
Le langage [PHP](https://www.php.net/) (PHP : Hypertexte Processor) a été développé en 1995 par le programmeur Rasmus Lerdorf. D'un langage écrit pour faciliter l'écriture de pages HTML répétitives, il s'est développé et est devenu l'un des langages les plus populaires pour le développement d'applications Web.  
Bien qu'il ait été longtemps critiqué pour certains aspects (il facilite l'écriture de code fragile voire non sûr et est peu efficace pour des sites ayant une très forte fréquentation), il reste aujourd'hui un langage incontournable en raison de sa simplicité d'utilisation. Les versions récentes apportent des améliorations notables en terme de sûreté et de performance.  
Comme Python, PHP est un langage interprété. L'interprète PHP lit les fichiers PHP et exécute le code qui s'y trouve.

### Exemple de programme PHP

#### Affichage de l'heure
Considérons le fichier [`heure.php`](Sites/Cours_Web/PHP/heure.php), dont le code est le suivant :

---
```php
<!DOCTYPE html>
<html lang = "fr">
   <head>
      <title>Quelle heure est-il?</title>
      <meta charset = "utf-8">
   </head>
   <body>
      Bonjour, il est : <?php echo date("H:i:s"); ?>
   </body>
</html>
```
---
Ce programme ressemble à une page HTML. Cependant, si ce fichier est déposé sur un serveur, par exemple : 

<div style="text-align: center">
    <a href="http://www.jdolivet.byethost13.com/NSI/Premiere/heure.php">http://www.jdolivet.byethost13.com/NSI/Premiere/heure.php</a> 
</div>
<br>

Les étapes suivantes se produisent :

1. Sur reception de la requête, le serveur Web passe le fichier `heure.php` à l'**interprète PHP**.
2. L'interprète copie automatiquement tous les caractères du fichier sur sa sortie standard, jusqu'à rencontrer la balise spéciale `<?php`.
3. Cette balise contient du **code PHP**, l'interprète exécute donc les instructions qu'il rencontre, ici `echo date("H:i:s");`. Cette instruction appelle la fonction prédéfinie [`date`](https://www.php.net/manual/fr/function.date) en lui passant en argument une chaîne de caractères. La fonction renvoie une chaîne de caractères représentant l'heure courante au format demandé (heures, minutes, secondes). Cette chaîne de caractères est passée, à son tour, à l'instruction [`echo`](https://www.php.net/manual/fr/function.echo), qui agit comme le `print` de Python et affiche son résultat sur la sortie standard.
4. Après avoir rencontré la suite de caractères `?>`, l'interprète reprend sa copie du reste du fichier HTML sur la sortie standard.
5. Le serveur Web récupère la sortie standard de l'interprète PHP et la renvoie au navigateur client comme réponse à la requête.

Ici, le client recevra une page HTML contenant l'heure courante (du serveur). S'il recharge la page alors tout le cycle recommence, et une **nouvelle heure** est renvoyée par la fonction `date`.

#### Récupération des paramètres

PHP permet bien évidemment de récupérer les valeurs des paramètres de requête

---
**Code HTML**

---

```html
<form action = "age.php">
   Rentrez votre année de naissance :
   <input type = "text" name = "a">
   <button type = "submit">Envoyer</button>
</form>
```

---
**Affichage** 

---
<form action = "age.php">
   Rentrez votre année de naissance :
   <input type = "text" name = "a">
   <button type = "submit">Envoyer</button>
</form>

---

La cible de ce formulaire est le fichier [`age.php`](Sites/Cours_Web/PHP/age.php) : 

---

```
<?php
   $annee = date("Y");
   $age = $annee - $_GET["a"];
?>
<!DOCTYPE html>
<html lang = "fr">
   <head>
      <title>Voici votre âge</title>
      <meta charset = "utf-8">
   </head>
   <body>
      Vous avez <?php echo $age; ?> ans cette année
   </body>
</html>
```

---

Le formulaire est accessible à cette adresse :

<div style="text-align: center">
    <a href="http://www.jdolivet.byethost13.com/NSI/Premiere/age.html">http://www.jdolivet.byethost13.com/NSI/Premiere/age.html</a> 
</div>

L'interprète PHP exécute le code se trouvant entre les balises `<?php` et `?>`. Les premières balises contiennent deux instructions. 
* La première définit une variable `$annee` (en PHP, les noms de variables commencent tous par `?`). Le contenu de la variable est le résultat de l'appel à la fonction `date` qui renvoie l'année courante comme une chaîne de caractères. 
* La seconde instruction définit une variable `$age`. En PHP, la variabale [`$_GET`](https://www.php.net/manual/fr/reserved.variables.get) est une variable globale prédéfinie. Elle contient un **dictionnaire** dont les clés sont les noms des paramètres de requête `GET` et qui sont associées aux valeurs de ces paramètres. 

Dans notre exemple, le formulaire HTML contient un champs dont l'attribut `name` vaut `a`. Lorsqu'on saisit une année (par exemple `1981`) et que l'on soumet le formulaire, le navigateur effectue une requête sur la ressource `age.php?a=1981`.  
Au début de son exécution, l'interprète PHP initialise le dictionnaire `$_GET` en associant la clé `"a"` à la valeur `1981` .  
La seconde instruction PHP effectue donc une soustraction entre l'année courante et `1981` et stocke le résultat dans la variable `$age` .  
La suite du fichier est copiée sur la sortie standard, jusqu'à ce que l'interprète évalue l'instruction `echo $age`, qui affiche juste la valeur de la variable sur la sortie standard. Le reste du fichier est ensuite copié puis renvoyé au client.  

PHP définit aussi un dictionnaire [`$_POST`](https://www.php.net/manual/fr/reserved.variables.post) qui s'utilise de la même façon et qui contient les valeurs des paramètres passés avec la méthode `POST` d'un formulaire.

### Gestion de la session
PHP permet d'utiliser la session HTTP pour implémenter un compteur de chargement de pages.  
Voici le programme [session.php](Sites/Cours_Web/PHP/session.php).

---
```
<?php
   session_start();
   if (!isset($_SESSION["compteur"])){
      $_SESSION["compteur"] = 0;
   }
   $_SESSION["compteur"] = $_SESSION["compteur"] + 1;
?>
<!DOCTYPE html>
<html lang = "fr">
   <head>
      <title>Chargement de la page</title>
      <meta charset = "utf-8">
   </head>
   <body>
      Page rechargée <?php echo $_SESSION["compteur"]; ?> fois.
   </body>
</html>
```

---
Le fichier déposé sur un serveur est accessible à cette adresse : 

<div style="text-align: center">
    <a href="http://www.jdolivet.byethost13.com/NSI/Premiere/session.php">http://www.jdolivet.byethost13.com/NSI/Premiere/session.php</a> 
</div>

Dans le code ci-dessus, l'appel à la fonction [`session_start()`](https://www.php.net/manual/fr/function.session-start) a un effet particulier.  
La première fois qu'un utilisateur visite la page, l'appel à cette fonction insère, dans les entêtes de la session un [cookie de session](Fichiers/Cookies/cookies_session.json). En interne, le serveur conserve un dictionnaire associé à ce cookie de session.  
Lors des visites suivantes, l'appel à `session_start()` relit le cookie de session envoyé par le navigateur et récupère le dictionnaire associé à ce cookie (le dictionnaire est stocké sur le serveur, c'est uniquement l'identifiant de session qui est stocké chez le client). Ce dictionnaire est accessible en PHP par la variable globale [`$_SESSION`](https://www.php.net/manual/fr/reserved.variables.session).  
Ici, on y teste la présence d'une clé `compteur`. Si elle est absente, on l'initialise à `0`. Sinon on prend la valeur et on l'incrémente, puis on la re-stocke dans le dictionnaire.  
Il suffit ensuite d'afficher sa valeur dans un message. A chaque fois que la page est rechargée, le compteur augmente.  
Lorsque l'utilisateur ferme son navigateur Web, le cookie de session est détruit.  
S'il se rend sur la page une nouvelle fois, tout se passera comme s'il ne l'avait jamais visitée et le décompte repartira de zéro.

### Exemple complet
Voici le programme [`table.php`](Sites/Cours_Web/PHP/table.php).

---
```
<?php
   if (isset($_GET["valeur"])){
      $valeur = $_GET["valeur"];
   }else{
      $valeur = 10;
   }
?>
<!DOCTYPE html>
<html lang = "fr">
   <head>
      <title>Table</title>
      <meta charset = "utf-8">
   </head>
   <body>
      <form action = "table.php">
         Valeur :
         <input type = "text" name = "valeur">
         <button type = "submit">Calculer</button>
      </form>
      <ul>
      <?php
         for ($i = 0; $i <= 10; $i = $i + 1){
            $style = "";
            if ($i % 2 != 0){
               $style = "background:gray; color:white";
            }
            echo "<li style = '" . $style . "'>";
            echo $valeur . " * " . $i . " = " . ($valeur * $i);
            echo "</li>\n";
         }
      ?>
      </ul>
   </body>
</html>
```
---
Le fichier déposé sur un serveur est accessible à cette adresse : 

<div style="text-align: center">
    <a href="http://www.jdolivet.byethost13.com/NSI/Premiere/table.php">http://www.jdolivet.byethost13.com/NSI/Premiere/table.php</a> 
</div>

Ce programme commence par récupérer un paramètre `GET` nommé `valeur`. Si ce dernier est présent, alors il est stocké dans une variable `$valeur`. S'il est absent (ce qui peut être le cas si on visite la page sans écrire explicitement le paramètre dans l'URL), alors on choisit la valeur `10` .  
Après ce code d'initialisation, on retrouve du code HTML classique : entête, balise `<body>` . Cette dernière contient un formulaire permettant de saisir la valeur dont la cible est **le fichier `table.php` lui même** .  
A la suite du formulaire, on retrouve à nouveau du code PHP. Ce dernier effectue une boucle `for` où la variable `$i` prend des valeurs de `0` à `10` inclus. Dans le corps de la boucle, on teste si `$i` est impair. Si c'est le cas, on définit une variable `$style` contenant une propriété CSS spécifiant que le fond est gris et le texte blanc. Enfin, on affiche dans la sortie le début de la balise `li`, puis son attribut `style`. En PHP, le [ . ](https://www.php.net/manual/fr/language.operators.string.php) permet de concaténer des chaînes. On affiche ensuite la multiplication courante puis on referme la balise.

Si on soumet une nouvelle valeur dans le formulaire et qu'on le soumet, la page se recharge, affichant la table correspondante.

### Autres langages
PHP n'est évidemment pas le seul langage de programmation Web côté serveur. A strictement parler, n'importe quel langage de programmation peut être utilisé côté serveur.  
A l'origine, les sites Web dynamiques étaient progrémmés en C. Le serveur Web passait les paramètres de requête sur l'entrée standard et le programme écrivait la page résultante sur la sortie standard.  
De nos jours, on utilise des langages modernes qui permettent de s'abstraire de considérations bas niveau. Outre PHP, le langage Python est un choix populaire pour la programmation côté serveur. Même s'il est possible de créer soit même un serveur Web en Python en utilisant uniquement la bibliothèque standard, la plupart des sites utilisent des bibliothèque spécialisées permettant la mise en forme facile de document HTML, la gestion simplifiée des paramètres et la gestion de session.  
D'autres choix populaires sont Java, Ruby, et, depuis quelques années, JavaScript, qui peut maintenant être utilisé aussi côté serveur.

## Exercices

### Exercice 1
De nombreux sites avec authentification permettent de cocher une case **se souvenir de moi**.  
Si cette case est cochée, l'utilisateur n'a plus besoin de s'authentifier pendant un mois, tant qu'il utilise le même navigateur.  
Comment est implémentée cette fonctionnalité?

### Exercice 2
Il est courant, lorsqu'on navigue sur un site où l'on s'est authentifié, qu'après plusieurs minutes sans utilisation, toute interaction avec le site affiche le message **votre session a expiré, veuillez vous reconnecter**.  
Comment peut-on expliquer ce phénomène?

### Exercice 3
Lorsqu'on navigue vers un site Web, ce dernier peut créer un identifiant de session qui identifie de manière unique un client. Cet identifiant est stocké dans un cookie sur le navigateur du client.  
Que se passerait-il si un serveur Web choisissait comme identifiant de session un simple compteur entier, incrémenté pour chaque nouveau visiteur?  

Par exemple lors de la première visite d'Alice, le site lui donne l'identifiant de session 1. Puis quand Bob visite, il obtient l'identifiant 2. Puis Charlie, l'identifiant 3. Puis quand Alice revient de nouveau, elle envoie le cookie `ID=1` et peut poursuivre sa session

### Exercice 4
On reprend l'exercice suivant : 

*Créer un fichier HTML contenant un formulaire, contenant lui-même deux champs de saisie de texte étiquetés `°C` et `°F` ainsi qu'un bouton de soumission.  
Ne pas mettre d'attribut `action` dans le formulaire, mais donner un nom (attribut `name`) et un attribut `id` aux deux champs de saisie.  
Donner un `id` au bouton.*

Le transformer en un fichier `conv.php` de la manière suivante :
1. Rajouter `conv.php` comme cible du formulaire.
2. Récupérer les valeurs de paramètre `GET c`.
3. `c` représente des degrés Celcius. Calculer la valeur correspondante en degrés Fahrenheit.
4. Utiliser les deux valeurs pour intialiser les attributs `value` des deux champs de texte.

On rappele que $F = C \times \dfrac{9}{5} +32$ où F est une température en degrés Fahrenheit et C un température en degrés Celcius.

### Exercice 5
On reprend l'exercice suivant : 

*Créer un fichier HTML contenant un formulaire contenant lui même trois champs de saisie de texte étiqueté `R`, `V` et `B` respectivement, ainsi qu'un bouton de soumission et un `div` d'`id` `"couleur"` en dessous du formulaire.  
Ajouter un élément `style` dans l'entête HTML pour que le `div` ait une largeur et une hauteur de 100 points et une bordure noire continue.  
Ne pas mettre d'attribut `action` dans le formulaire, mais donner un nom (attribut `name`) et un attribut `id` aux trois champs de saisie.*

Le transformer en un fichier `couleur.php` de la manière suivante :
1. Récupérer les valeurs des trois paramètres `r`,`g`, `b`. Ces valeurs sont supposées être entre `0` et `255`.
2. Former une chaîne de caractères `"rgb(r,v,b)"` ou r, v et b sont les trois valeurs lues précédemment.
3. Utiliser cette chaîne comme valeur de l'attribut propriété CSS `background` pour changer le fond du `div` se trouvant dans la page.

## Liens :
* Document accompagnement Eduscol : [Interaction client-serveur](https://cache.media.eduscol.education.fr/file/NSI/77/1/RA_Lycee_G_NSI_ihm_interaction_client_serveur_1170771.pdf)
* [https://www.php.net/](https://www.php.net/)
* PHP Tutorial : [W3Schools](https://www.w3schools.com/php/default.asp)
* Interstices : [Outils informatiques pour la protection de la vie privée](https://interstices.info/outils-informatiques-pour-la-protection-de-la-vie-privee/)
* CNIL : [Cookies et traceurs : que dit la loi ?](https://www.cnil.fr/fr/cookies-et-traceurs-que-dit-la-loi)
* CNIL : [Une petite histoire du cookie](https://linc.cnil.fr/fr/une-petite-histoire-du-cookie)
* LINC : [Dossier Cookies](https://linc.cnil.fr/fr/dossier-cookies)
* CNIL : [Le règlement général sur la protection des données - RGPD](https://www.cnil.fr/fr/reglement-europeen-protection-donnees)